Assignment 1 Part 1 - Hayden Barker

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os

def cnn_model_fn(features, labels, mode):
#"""Model function for CNN."""
# Input Layer
# Reshape X to 4-D tensor: [batch_size, width, height, channels]
# MNIST images are 28x28 pixels, and have one color channel
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
# Convolutional Layer #1
# Computes 4 features using a 5x5 filter with ReLU activation.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 28, 28, 1]
# Output Tensor Shape: [batch_size, 28, 28, 4]
  conv1 = tf.layers.conv2d(inputs=input_layer, filters=4, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
# Pooling Layer #1
# First max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 28, 28, 4]
# Output Tensor Shape: [batch_size, 14, 14, 4]
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
# Convolutional Layer #2
# Computes 8 features using a 5x5 filter.
# Padding is added to preserve width and height.
# Input Tensor Shape: [batch_size, 14, 14, 4]
# Output Tensor Shape: [batch_size, 14, 14, 8]
  conv2 = tf.layers.conv2d(inputs=pool1, filters=8, kernel_size=[5, 5], padding="same", activation=tf.nn.relu)
# Pooling Layer #2
# Second max pooling layer with a 2x2 filter and stride of 2
# Input Tensor Shape: [batch_size, 14, 14, 8]
# Output Tensor Shape: [batch_size, 7, 7, 8]
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
# Flatten tensor into a batch of vectors
# Input Tensor Shape: [batch_size, 7, 7, 8]
# Output Tensor Shape: [batch_size, 7 * 7 * 8]
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 8])
# Dense Layer
# Densely connected layer with 10 neurons
# Input Tensor Shape: [batch_size, 7 * 7 * 64]
# Output Tensor Shape: [batch_size, 10]
  logits = tf.layers.dense(inputs=pool2_flat, units=10)
    
  predictions = {
# Generate predictions (for PREDICT and EVAL mode)
"classes": tf.argmax(input=logits, axis=1),
# Add `softmax_tensor` to the graph. It is used for PREDICT and by the
# `logging_hook`.
"probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }
  
  print("Trainable Parameters : ", np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()]))
  

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
  
# Calculate Loss (for both TRAIN and EVAL modes)
# Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN: 
    loss = tf.losses.mean_squared_error(labels = labels, predictions = logits)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
  
# Add evaluation metrics (for EVAL mode)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
  eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


def main():
  # get the data
  mnist = input_data.read_data_sets("MNIST_data/")
  eval_data = mnist.test.images
  eval_labels = np.asarray(mnist.test.labels, dtype=np.int32) 
  # create a session
  sess = tf.InteractiveSession()
  sess.run(tf.global_variables_initializer())
  # load the checkpoint
  # example filepath for checkpoint : /Users/haydenbarker/Desktop/Assignment1
  filepath = input('Enter the filepath for the checkpoint : ')
  #/Users/haydenbarker/Desktop/tensorflow-mnist-cnn-master/Part1
  mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir = filepath)
  # evaluate the model
  eval_input_fn = tf.estimator.inputs.numpy_input_fn(x={"x": eval_data}, y=eval_labels, num_epochs=1, shuffle=False)
  eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)  
    

if __name__ == "__main__":
    main()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Enter the filepath for the checkpoint : /Users/haydenbarker/Desktop/Assignment1
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/Users/haydenbarker/Desktop/Assignment1', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1206da748>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Trainable Parameters :  4842
INFO:tensorflow:Starting evaluation at 2018-03-07-23:46:09
INFO:tensorflow:Restoring parameters fr

Number of trainable parameters in the model is : 4842
{'accuracy': 0.9884, 'loss': 0.0365276, 'global_step': 20000}